In [1]:
# compenency query is for fasc diam, nerv diam, and lateriality

In [2]:
import pandas as pdfdsadaasdfd
import uuid

from sqlalchemy import inspect
from sqlalchemy.orm import Session
from sqlalchemy.inspection import inspect
from sqlalchemy.exc import IntegrityError

from quantdb.ingest import Queries, InternalIds
from quantdb.client import get_session
from sqlalchemy.sql.elements import ClauseElement
from quantdb.ingest_tabular.models import (
    # Ingest,
    # Unit,
    Objects,
    Units,
    Aspects,
    DescriptorsInst,
    DescriptorsQuant,
    ObjDescInst,
    ObjDescQuant,
    ValuesInst,
    ValuesQuant,
)

session = get_session(echo=False)

object_ = Objects(
    id="55c5b69c-test-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# ingest = Ingest(session=session)
unit = Units(
    label="test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/test",
)

aspect = Aspects(
    label="diameter-test",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test",
)

desc_inst = DescriptorsInst(
    label="nerve-volume-test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    description=None,
)

desc_quant = DescriptorsQuant(
    shape="scalar",
    label="nerve-volume-test3",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
)

print(list(desc_quant.__mapper__.relationships)[0].direction)
getattr(desc_quant, "aspects", [])

RelationshipDirection.MANYTOONE


/Users/tmsincomb/Dropbox (Personal)/repos/quantitative-db/quantdb/ingest_tabular/models.py:1424: SAWarning: relationship 'ObjDescInst.values_cat' will copy column obj_desc_inst.desc_inst to column values_cat.desc_inst, which conflicts with relationship(s): 'DescriptorsInst.values_cat' (copies descriptors_inst.id to values_cat.desc_inst). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="values_cat"' to the 'ObjDescInst.values_cat' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated 

In [3]:
def object_as_dict(obj):
    return {
        c.key: getattr(obj, c.key)
        for c in inspect(obj).mapper.column_attrs
        if getattr(obj, c.key)
    }


def get_or_create(session, obj, back_populate=None):
    model = obj.__class__
    data = object_as_dict(obj)
    instance = session.query(model).filter_by(**data).one_or_none()
    if instance:
        return instance
    else:
        params = {
            k: v for k, v in data.items() if not isinstance(v, ClauseElement)
        }
        instance = model(**params)
        try:
            session.add(instance)
            session.commit()
            if back_populate:
                for attr, value in back_populate.items():
                    setattr(instance, attr, value)
                session.commit()
        except Exception:
            session.rollback()
            instance = session.query(model).filter_by(**data).one()
            return instance
        else:
            return instance


def back_populate_tables(db: Session, obj) -> object:
    """
    Recursively back-populates objects in the database,
    retrieving existing objects instead of failing on duplicates.
    Iterates through all relationships of the object for both
    initial population and recursive calls.
    Includes a complete rollback if any commit fails.
    Filters by all primary key columns of the object.
    Handles cases where the child object is not a valid ORM instance.

    Args:
        db: The database session.
        obj: The ORM model object.

    Returns:
        The ORM model object, with its related objects back-populated.
    """

    mapper = obj.__mapper__  # Get the mapper for the object
    print(obj)
    try:
        # Iterate through all relationships of the object
        for relationship_prop in mapper.relationships:
            if (
                relationship_prop.direction.name == "MANYTOONE"
                or relationship_prop.direction.name == "MANYTOMANY"
            ):
                parent = getattr(obj, relationship_prop.key)
                if parent:
                    filter_criteria = object_as_dict(parent)
                    existing_obj = (
                        db.query(type(parent))
                        .filter_by(**filter_criteria)
                        .first()
                    )
                    if existing_obj:
                        for key, value in object_as_dict(existing_obj).items():
                            if hasattr(parent, key):
                                setattr(parent, key, value)
                    print("populated", object_as_dict(parent))
                    setattr(obj, relationship_prop.key, parent)
                    # db.add(parent)
                    # db.commit()
                    back_populate_tables(db, parent)

        filter_criteria = object_as_dict(obj)
        print("filter using", filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        print("is existing?", existing_obj)
        if not existing_obj:
            db.merge(obj)
            db.commit()
        else:
            for key, value in object_as_dict(existing_obj).items():
                if hasattr(obj, key):
                    setattr(obj, key, value)
        print("after", object_as_dict(existing_obj))

    except IntegrityError as e:
        db.rollback()  # Rollback the transaction in case of an error
        print(f"Error during commit: {e}")
        raise  # Re-raise the exception to indicate failure

    return obj  # Return the object with its related objects back-populated

In [4]:
# Params
object = Objects(
    id="N:dataset:99a9a99a-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# back_populate_tables(session, object)

In [5]:
# Domain
unit = Units(
    label="test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/test",
)

aspect = Aspects(
    label="diameter-test-backpop4",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test-backpop4",
)

desc_inst = DescriptorsInst(
    label="nerve-volume-test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    description=None,
)

desc_quant = DescriptorsQuant(
    shape="scalar",
    label="nerve-volume-test8",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)


# getattr(desc_quant, "aspects")
# list(desc_quant.aspects.__mapper__.relationships)
# back_populate_tables(session, desc_quant)
# print("here", aspect.id)
# print("here", desc_quant.aspects)

In [6]:
# Value
values_inst = ValuesInst(
    type="sample",
    id_sub="sub-test",
    id_formal="sam-l-seg-c7-A-level-test",
    local_identifier=None,
    id_sam="sam-l-seg-c7-A-level-test",
    # descriptors_inst=desc_inst,
    # dataset=object.id,
    objects=object,
)
#     "id": 2,
#     "value": 0.11059190031152646,
#     "object": "38215e04-e8ab-4a55-8291-46839f96ed1b",
#     "desc_inst": 15,
#     "desc_quant": 3,
#     "instance": 20,
#     "orig_value": null,
#     "orig_units": null,
#     "value_blob": 0.11059190031152646
#   },
values_quant = ValuesQuant(
    value=0.5,
    objects=object,
    orig_value=None,
    orig_units=None,
    descriptors_inst=desc_inst,
    descriptors_quant=desc_quant,
    values_inst=values_inst,
)

with session.no_autoflush:
    back_populate_tables(session, values_quant)

populated {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
filter using {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is existing? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x148db71a0>
after {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
populated {'id': 34, 'shape': 'scalar', 'label': 'nerve-volume-test8', 'aggregation_type': 'instance', 'aspect': 29, 'domain': 34}
populated {'id': 29, 'label': 'diameter-test-backpop4', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test-backpop4'}
filter using {'id': 29, 'label': 'diameter-test-backpop4', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test-backpop4'}
is existing? <quantdb.ingest_tabular.models.Aspects object at 0x148de5fd0>
afte

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "value_blob" of relation "values_quant" violates not-null constraint
DETAIL:  Failing row contains (649, 0.5, 99a9a99a-a5b8-4881-a105-e4048af26fa5, 34, 34, 227, null, null, null).

[SQL: INSERT INTO values_quant (value, object, desc_inst, desc_quant, instance, orig_value, orig_units) VALUES (%(value)s, %(object)s::UUID, %(desc_inst)s, %(desc_quant)s, %(instance)s, %(orig_value)s, %(orig_units)s) RETURNING values_quant.id]
[parameters: {'value': 0.5, 'object': UUID('99a9a99a-a5b8-4881-a105-e4048af26fa5'), 'desc_inst': 34, 'desc_quant': 34, 'instance': 227, 'orig_value': None, 'orig_units': None}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
values_quant = ValuesQuant(
    value=0.5,
    objects=object,
    orig_value=None,
    orig_units=None,
    descriptors_inst=desc_inst,
    descriptors_quant=desc_quant,
    values_inst=values_inst,
)
desc_inst.id = 1
values_quant.descriptors_inst = desc_inst
values_quant.desc_inst

1

In [ ]:
object_as_dict(values_quant)

{'value': 0.5, 'desc_inst': 1}

In [ ]:
# ingest.insert_objects(
#     objects=[
#         Objects(id="55c5b69c-a5b8-4881-a105-e4048af26fa5", id_type="dataset"),
#     ]
# )

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df

NameError: name 'pd' is not defined

# Testing values_quant for NFasc

# dream ingest

In [ ]:
# ingest.insert_descriptors_quant(
#     descriptor_quants=[
#         DescriptorQuant(
#             label="fascicle cross section diameter um min",
#             domain=26,  # "fiber-cross-section" # TODO: label to ID
#             aspect=5,  # "diameter",  # TODO: label to ID
#             unit=4,  #  pixel",  # TODO: label to ID
#             aggregation_type="min",
#         ),
#     ]
# )

# # dream ingest
# csv = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
# ingest.insert_descriptors_quant(**csv.to_dict(orient="records"))

[autoreload of quantdb.ingest_tabular.models failed: Traceback (most recent call last):
  File "/Users/tmsincomb/miniforge3/envs/quantdb/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/tmsincomb/miniforge3/envs/quantdb/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/tmsincomb/miniforge3/envs/quantdb/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/tmsincomb/miniforge3/envs/quantdb/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 335, in update_class
    if (old_obj == new_obj) is True:
        ^^^^^^^^^^^^^^^^^^
  File "/Users/tmsincomb/miniforge3/envs/quantdb/lib/python3.12/site-packages/sqlalchemy/sql/operators.py", line 582, in __eq__
    return self.operate(eq, other)
           ^^^^^^^^^^^^^^^^^^^^^^^
 